In [1]:
!pip install plotly
!pip install numpy
!pip install matplotlib
!pip install statsmodels
!pip install pandas
!pip install sklearn
!pip install seaborn
!pip install yellowbrick
!pip install scipy

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import csv
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import Ridge
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot
from yellowbrick.regressor import ResidualsPlot
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
from scipy.stats import normaltest

In [3]:
trainSetGiven = pd.read_csv("data/train.csv")
trainSetGiven.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
#We clean the data
trainSetGiven = trainSetGiven.drop(columns=['Id'])


In [5]:
#Get only numeric
dataSetCompleteNumeric = trainSetGiven._get_numeric_data().dropna(how='any')
lenP = len(dataSetCompleteNumeric['SalePrice'])
# dataSetCompleteNumeric['es_caro'] = np.random.randn(lenP)
# dataSetCompleteNumeric['es_barato'] = np.random.randn(lenP)
# dataSetCompleteNumeric['es_medio'] = np.random.randn(lenP)

dataSetCompleteNumeric

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,349,0,0,0,0,0,0,2,2010,210000
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,60,0,0,0,0,2500,5,2010,266500
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,366,0,112,0,0,0,0,4,2010,142125


In [6]:
# We sperate to get valid sets
caros = dataSetCompleteNumeric[dataSetCompleteNumeric['SalePrice']>150000].copy()
# caros['cat'] = 'Caro'
caros['es_caro'] = 1
caros['es_barato'] = 0
caros['es_medio'] = 0
baratos = dataSetCompleteNumeric[dataSetCompleteNumeric['SalePrice']<80000].copy()
baratos['es_caro'] = 0
baratos['es_barato'] = 1
baratos['es_medio'] = 0
medio = dataSetCompleteNumeric[dataSetCompleteNumeric['SalePrice'] > 80000].copy()
medio = medio[medio['SalePrice'] < 150000]
medio['es_caro'] = 0
medio['es_barato'] = 0
medio['es_medio'] = 1
caros

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,es_caro,es_barato,es_medio
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,2,2008,208500,1,0,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,0,5,2007,181500,1,0,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,0,9,2008,223500,1,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,0,12,2008,250000,1,0,0
6,20,75.0,10084,8,5,2004,2005,186.0,1369,0,...,0,0,0,0,8,2007,307000,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,20,78.0,9262,8,5,2008,2009,194.0,0,0,...,0,0,0,0,5,2009,287090,1,0,0
1454,20,62.0,7500,7,5,2004,2005,0.0,410,0,...,0,0,0,0,10,2009,185000,1,0,0
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,0,8,2007,175000,1,0,0
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,0,2,2010,210000,1,0,0


In [7]:
#Proporciones caros
trainCaro = caros.sample(frac=0.65)
testCaro = caros.drop(trainCaro.index)

#Proporciones baratos
trainBarato = baratos.sample(frac=0.65)
testBarato = baratos.drop(trainBarato.index)

#Proporciones medio
trainMedio = medio.sample(frac=0.65)
testMedio = medio.drop(trainMedio.index)

# Proporcion 35, 65
train = trainCaro.append(trainMedio).append(trainBarato)
test  = testCaro.append(testMedio).append(testBarato)

#all data
datos = train.append(test)
salePrice = datos.pop("SalePrice")

In [8]:
#Plot config
plt.rcParams['figure.figsize'] = (15, 9)
plt.style.use('ggplot')

# Separaremos valores y calcularemos con Los diferentes Y
# Calcularemos los Caros

In [9]:

import random

#Medir si es caro o
c = datos.copy()
y_caro = c.pop("es_caro")
y_barato = c.pop("es_barato")
y_medio = c.pop("es_medio")
y = y_caro #La dicotómicas respuesta cara
X = c #El resto de los datos
random.seed(123)
y

172     1
704     1
910     1
647     1
959     1
       ..
874     0
915     0
935     0
1279    0
1380    0
Name: es_caro, Length: 1116, dtype: int64

In [10]:
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.3,train_size=0.7)

# Haciendo regresion con Caros

In [11]:
from sklearn.metrics import confusion_matrix
logReg = LogisticRegression(solver='liblinear')
logReg.fit(X_train,y_train)
y_pred = logReg.predict(X_test)
y_proba = logReg.predict_proba(X)[:,1]
cm = confusion_matrix(y_test,y_pred)

## Medir presicion Caros

In [12]:
from sklearn.metrics import accuracy_score ,precision_score,recall_score,f1_score
accuracy=accuracy_score(y_test,y_pred)
precision =precision_score(y_test, y_pred,average='micro')
recall =  recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test,y_pred,average='micro')
print('Matriz de confusión para detectar casas con valor cara\n',cm)
print('Accuracy: ',accuracy)

Matriz de confusión para detectar casas con valor cara
 [[121  11]
 [ 17 186]]
Accuracy:  0.9164179104477612


# Resultados caros
Dentro de la matriz de confusión que nos mide los verdaderos correctos y los falsos correctos y viceversa podemos ver que tuvo una precision del 91.64%

# Calcularemos los baratos


In [13]:
y = y_barato #La variable dicotómicas barata
X = c #El resto de los datos

In [14]:
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.3,train_size=0.7)

# Haciendo regresion con Baratos

In [15]:
logReg = LogisticRegression(solver='liblinear')
logReg.fit(X_train,y_train)
y_pred = logReg.predict(X_test)
y_proba = logReg.predict_proba(X)[:,1]
cm = confusion_matrix(y_test,y_pred)

In [16]:
accuracy=accuracy_score(y_test,y_pred)
precision =precision_score(y_test, y_pred,average='micro')
recall =  recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test,y_pred,average='micro')
print('Matriz de confusión para detectar casas con valor barato\n',cm)
print('Accuracy: ',accuracy)

Matriz de confusión para detectar casas con valor barato
 [[327   1]
 [  5   2]]
Accuracy:  0.982089552238806


# Resultados baratos
En este modelo tuvimos una precision del 98.2% seindo excelente casi perfecto

# Calcularemos los medios

In [17]:
y = y_medio #La variable dicotómicas medio
X = c #El resto de los datos

In [18]:
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.3,train_size=0.7)

# Haciendo regresion con medios

In [19]:
logReg = LogisticRegression(solver='liblinear')
logReg.fit(X_train,y_train)
y_pred = logReg.predict(X_test)
y_proba = logReg.predict_proba(X)[:,1]
cm = confusion_matrix(y_test,y_pred)

In [20]:
accuracy=accuracy_score(y_test,y_pred)
precision =precision_score(y_test, y_pred,average='micro')
recall =  recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test,y_pred,average='micro')
print('Matriz de confusión para detectar casas con valor medio\n',cm)
print('Accuracy: ',accuracy)

Matriz de confusión para detectar casas con valor medio
 [[186  24]
 [ 14 111]]
Accuracy:  0.8865671641791045


# Resultados medios
En este modelo tuvimos una precision del 88.65% seindo excelente mas sin embargo no fue mejor que las casas baratas

# Resumen
Se observaron buenos resultados en las 3 tanto las medias como las caras se corrieron vairas veces no se diria que una de esos modelos es mejor pero las baratas siempre sacaron ventaja.

# Cual se tardo mas?
Ni una realmente todas parecieron correr igual de rapidos tal vez demoro un poco mas las medias

# ¿Cuál se equivocó más?
Se corrieron varias veces y tanto las caras como las medias rondaron entre 87% y 92% asi que no nos animariamos a decir cual de esas fallo menos. Pero tiene sentido ya que este grupo es mayor a las baratas.

# ¿Cuál se equivocó menos? y ¿por qué?
Se equivocaron menos las baratas porque su rango era mas delimitado reduciendo asi el margen de error y teniendo sets mas precisos.
En otras palabras la variable categorica que se tomo fue mejor otorgada.
